# 📘 Full System Stress Test

This notebook exercises the full PINN Greeks stack — environment, data integrity, baselines, training, 
evaluation, and diagnostics. Run the cells top-to-bottom whenever you need a regression-style sanity check.

**Sections**
1. Environment setup and project paths
2. Configuration and reproducibility controls
3. Dataset availability and validation
4. Baseline estimators vs analytic Greeks
5. PINN training / checkpoint loading
6. Quantitative evaluation on validation data
7. PDE residual and surface diagnostics
8. Summary + JSON export for logging


In [1]:

# 0) Environment & folders
import os, sys, json, math, random, time
from pathlib import Path
from datetime import datetime
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 120

try:
    import torch
    TORCH_OK = True
except Exception as e:
    TORCH_OK = False
    print("PyTorch import failed:", e)

NOTEBOOK_CWD = Path.cwd().resolve()
CANDIDATES = [NOTEBOOK_CWD, NOTEBOOK_CWD.parent, NOTEBOOK_CWD.parent.parent]
BASE = None
for candidate in CANDIDATES:
    if (candidate / "src").is_dir():
        BASE = candidate
        break

if BASE is None:
    raise RuntimeError(f"Could not locate project root from {NOTEBOOK_CWD}")

print(f"Project root: {BASE}")
if str(BASE) not in sys.path:
    sys.path.insert(0, str(BASE))

for d in [
    'src', 'src/utils', 'src/models', 'src/baselines',
    'data', 'results', 'figures', 'figures/data_exploration',
    'figures/training_curves', 'figures/residual_heatmaps', 'figures/final_results',
    'figures/stress_test'
]:
    (BASE / d).mkdir(parents=True, exist_ok=True)

DATA_DIR = BASE / 'data'
FIGURES_DIR = BASE / 'figures'
RESULTS_DIR = BASE / 'results'
REPORTS_DIR = BASE / 'reports'
REPORTS_DIR.mkdir(exist_ok=True)

SEED = 123
np.random.seed(SEED)
random.seed(SEED)

if TORCH_OK:
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)
    DEFAULT_DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
else:
    DEFAULT_DEVICE = 'cpu'

print({
    'python': sys.version.split()[0],
    'numpy': np.__version__,
    'pandas': pd.__version__,
    'matplotlib': matplotlib.__version__,
    'torch': torch.__version__ if TORCH_OK else None,
    'device_default': DEFAULT_DEVICE,
})


Project root: /Users/amv10802/Documents/Neural-PDE-Option-Greeks
{'python': '3.11.7', 'numpy': '2.3.3', 'pandas': '2.3.3', 'matplotlib': '3.10.7', 'torch': '2.8.0', 'device_default': 'cpu'}


In [2]:

# 1) Configuration (edit values here for quick experiments)
CONFIG = {
    'train_epochs': 10,
    'batch_size': 4096,
    'learning_rate': 1e-3,
    'warmup_steps': 500,
    'grad_clip': 1.0,
    'baseline_samples': 1000,
    'mc_paths': 20000,
    'mc_eval_points': 25,
    'num_val_samples': 5000,
    'residual_grid': 40,
    'load_existing_checkpoint': True,
    'save_checkpoint': False,
    'checkpoint_name': 'pinn_checkpoint.pt',
    'device': DEFAULT_DEVICE,
}

CONFIG


{'train_epochs': 10,
 'batch_size': 4096,
 'learning_rate': 0.001,
 'warmup_steps': 500,
 'grad_clip': 1.0,
 'baseline_samples': 1000,
 'mc_paths': 20000,
 'mc_eval_points': 25,
 'num_val_samples': 5000,
 'residual_grid': 40,
 'load_existing_checkpoint': True,
 'save_checkpoint': False,
 'checkpoint_name': 'pinn_checkpoint.pt',
 'device': 'cpu'}

In [3]:

# 2) Dataset availability + loading (train/val)
from src.data import generate_dataset

train_path = DATA_DIR / 'synthetic_train.npy'
val_path = DATA_DIR / 'synthetic_val.npy'

if not train_path.exists() or not val_path.exists():
    print("Synthetic dataset not found — generating via src.data.generate_dataset()")
    generate_dataset(output_dir=DATA_DIR)
else:
    print("Found existing synthetic datasets.")

train_data = np.load(train_path)
val_data = np.load(val_path)

columns = ['S', 't', 'sigma', 'V']
train_df = pd.DataFrame(train_data, columns=columns)
val_df = pd.DataFrame(val_data, columns=columns)

print({
    'train_samples': len(train_df),
    'val_samples': len(val_df),
    'train_path': str(train_path),
    'val_path': str(val_path),
})


Found existing synthetic datasets.
{'train_samples': 1000000, 'val_samples': 100000, 'train_path': '/Users/amv10802/Documents/Neural-PDE-Option-Greeks/data/synthetic_train.npy', 'val_path': '/Users/amv10802/Documents/Neural-PDE-Option-Greeks/data/synthetic_val.npy'}


In [4]:

# 3) Dataset diagnostics
summary_cols = ['S', 't', 'sigma', 'V']
train_summary = train_df[summary_cols].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])
val_summary = val_df[summary_cols].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])

print("Train summary:")
print(train_summary)
print("Validation summary:")
print(val_summary)

nan_train = np.isnan(train_data).sum(axis=0)
nan_val = np.isnan(val_data).sum(axis=0)
print("NaN counts (train):", dict(zip(summary_cols, nan_train)))
print("NaN counts (val):", dict(zip(summary_cols, nan_val)))

# Stress test boundary conditions
S_min, S_max = train_df['S'].min(), train_df['S'].max()
print(f"S range: {S_min:.2f} → {S_max:.2f}")
print(f"t range: {train_df['t'].min():.4f} → {train_df['t'].max():.4f}")
print(f"sigma range: {train_df['sigma'].min():.4f} → {train_df['sigma'].max():.4f}")


Train summary:
                    S               t           sigma             V
count  1000000.000000  1000000.000000  1000000.000000  1.000000e+06
mean       110.060208        1.003963        0.325066  3.382769e+01
std         51.946403        0.574145        0.158812  3.422119e+01
min         20.000093        0.010001        0.050001  0.000000e+00
1%          21.804753        0.029708        0.055495  3.280156e-88
5%          29.087468        0.108898        0.077477  6.087755e-14
50%        110.100075        1.002917        0.325225  2.256246e+01
95%        191.048427        1.900753        0.572594  9.711937e+01
99%        198.206258        1.979981        0.594453  1.055692e+02
max        199.999697        1.999999        0.599999  1.189780e+02
Validation summary:
                   S              t          sigma             V
count  100000.000000  100000.000000  100000.000000  1.000000e+05
mean      110.348122       1.005956       0.324366  3.396204e+01
std        51.884131  

In [5]:

# 4) Baseline estimators vs analytic Greeks
from src.utils.black_scholes import bs_price, bs_greeks
from src.baselines.finite_difference import finite_diff_greeks
from src.baselines.monte_carlo import mc_pathwise_delta

K = 100.0
T = 2.0
r = 0.05
snapshot_t = 1.0
snapshot_sigma = 0.2

S_eval = np.linspace(20, 200, CONFIG['baseline_samples'])
fd_delta, fd_gamma = finite_diff_greeks(S_eval, K=K, T=T, t=snapshot_t, sigma=snapshot_sigma, r=r)
analytic = bs_greeks(S_eval, K, T, snapshot_t, snapshot_sigma, r)
analytic_delta = analytic['delta']
analytic_gamma = analytic['gamma']

fd_delta_mae = np.mean(np.abs(fd_delta - analytic_delta))
fd_gamma_mae = np.mean(np.abs(fd_gamma - analytic_gamma))

mc_points = np.linspace(40, 160, CONFIG['mc_eval_points'])
mc_results = []
for idx, S0 in enumerate(mc_points):
    est = mc_pathwise_delta(S0, K=K, T=snapshot_t, r=r, sigma=snapshot_sigma, N=CONFIG['mc_paths'], seed=idx)
    target = bs_greeks(S0, K, T, snapshot_t, snapshot_sigma, r)['delta']
    mc_results.append({'S': S0, 'mc_delta': est, 'analytic_delta': target, 'abs_err': abs(est - target)})
mc_df = pd.DataFrame(mc_results)

print(f"Finite-difference delta MAE: {fd_delta_mae:.6f}")
print(f"Finite-difference gamma MAE: {fd_gamma_mae:.6f}")
print("Monte Carlo delta summary:")
print(mc_df.describe())


Finite-difference delta MAE: 0.000000
Finite-difference gamma MAE: 0.000000
Monte Carlo delta summary:
                S   mc_delta  analytic_delta    abs_err
count   25.000000  25.000000       25.000000  25.000000
mean   100.000000   0.539672        0.539240   0.001780
std     36.799004   0.412345        0.412186   0.001857
min     40.000000   0.000000        0.000012   0.000012
25%     70.000000   0.077521        0.075875   0.000417
50%    100.000000   0.642709        0.636831   0.001321
75%    130.000000   0.950405        0.951726   0.002056
max    160.000000   0.996094        0.996533   0.006395


In [6]:

# 5) Train or load PINN model
if not TORCH_OK:
    model = None
    training_log = []
    print('PyTorch not available — skipping PINN training.')
else:
    from src.models import PINNModel
    from src.losses import pinn_loss
    from src.train import load_data

    device = torch.device(CONFIG['device'])
    checkpoint_path = RESULTS_DIR / CONFIG['checkpoint_name']
    model = PINNModel().to(device)
    training_log = []

    if CONFIG['load_existing_checkpoint'] and checkpoint_path.exists():
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))
        print(f"Loaded checkpoint: {checkpoint_path}")
    else:
        loader = load_data(path=train_path, batch_size=CONFIG['batch_size'])
        opt = torch.optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])
        base_lr = CONFIG['learning_rate']
        warmup = max(0, CONFIG['warmup_steps'])
        global_step = 0

        for epoch in range(CONFIG['train_epochs']):
            epoch_loss = 0.0
            epoch_price = 0.0
            epoch_pde = 0.0
            epoch_reg = 0.0
            steps = 0
            model.train()
            for batch in loader:
                S_b, t_b, sigma_b, V_b = [x.to(device) for x in batch]
                opt.zero_grad()
                loss, (L_price, L_PDE, L_reg) = pinn_loss(model, S_b, t_b, sigma_b, V_b)
                if warmup > 0:
                    lr_scale = min(1.0, (global_step + 1) / warmup)
                    for g in opt.param_groups:
                        g['lr'] = base_lr * lr_scale
                loss.backward()
                if CONFIG['grad_clip'] and CONFIG['grad_clip'] > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['grad_clip'])
                opt.step()

                epoch_loss += loss.item()
                epoch_price += L_price.item()
                epoch_pde += L_PDE.item()
                epoch_reg += L_reg.item()
                steps += 1
                global_step += 1

            log_entry = {
                'epoch': epoch + 1,
                'loss': epoch_loss / steps,
                'price': epoch_price / steps,
                'pde': epoch_pde / steps,
                'reg': epoch_reg / steps,
                'lr': opt.param_groups[0]['lr'],
            }
            training_log.append(log_entry)
            print(f"Epoch {log_entry['epoch']:03d} | loss={log_entry['loss']:.6f} | "
                  f"price={log_entry['price']:.6f} | pde={log_entry['pde']:.6f} | reg={log_entry['reg']:.6f}")

        if CONFIG['save_checkpoint']:
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Saved checkpoint to {checkpoint_path}")

    if training_log:
        training_df = pd.DataFrame(training_log)
        display(training_df)


Epoch 001 | loss=1357.592095 | price=1355.037248 | pde=2.554317 | reg=0.000534
Epoch 002 | loss=1303.461871 | price=1293.289988 | pde=10.171536 | reg=0.000351
Epoch 003 | loss=1280.109765 | price=1279.041850 | pde=1.067637 | reg=0.000284
Epoch 004 | loss=1238.052212 | price=1237.904972 | pde=0.147035 | reg=0.000204
Epoch 005 | loss=1194.792098 | price=1194.702432 | pde=0.089504 | reg=0.000150
Epoch 006 | loss=1185.102789 | price=1184.995081 | pde=0.107610 | reg=0.000075
Epoch 007 | loss=1183.288234 | price=1183.165130 | pde=0.123076 | reg=0.000052
Epoch 008 | loss=1198.115457 | price=1198.002117 | pde=0.113275 | reg=0.000063
Epoch 009 | loss=1186.720478 | price=1186.577019 | pde=0.143441 | reg=0.000045
Epoch 010 | loss=1186.528250 | price=1186.397155 | pde=0.131061 | reg=0.000043


,epoch,loss,price,pde,reg,lr
0,1,1357.592095,1355.037248,2.554317,0.000534,0.00049
1,2,1303.461871,1293.289988,10.171536,0.000351,0.00098
2,3,1280.109765,1279.041850,1.067637,0.000284,0.00100
3,4,1238.052212,1237.904972,0.147035,0.000204,0.00100
4,5,1194.792098,1194.702432,0.089504,0.000150,0.00100
5,6,1185.102789,1184.995081,0.107610,0.000075,0.00100
6,7,1183.288234,1183.165130,0.123076,0.000052,0.00100
7,8,1198.115457,1198.002117,0.113275,0.000063,0.00100
8,9,1186.720478,1186.577019,0.143441,0.000045,0.00100
9,10,1186.528250,1186.397155,0.131061,0.000043,0.00100


In [7]:

# 6) Quantitative evaluation on validation data
if not TORCH_OK or model is None:
    evaluation_summary = None
    print('Skipping evaluation step (model unavailable).')
else:
    model.eval()
    device = next(model.parameters()).device
    N_eval = min(CONFIG['num_val_samples'], len(val_df))
    idx = np.random.choice(len(val_df), size=N_eval, replace=False)
    batch = val_data[idx]
    S_np = batch[:, 0]
    t_np = batch[:, 1]
    sigma_np = batch[:, 2]
    target_price = batch[:, 3]

    S = torch.tensor(S_np, dtype=torch.float32, device=device, requires_grad=True)
    t = torch.tensor(t_np, dtype=torch.float32, device=device, requires_grad=True)
    sigma = torch.tensor(sigma_np, dtype=torch.float32, device=device, requires_grad=True)

    inputs = torch.stack([S, t, sigma], dim=1)
    inputs.requires_grad_(True)
    preds = model(inputs).squeeze()

    ones = torch.ones_like(preds)
    grad = torch.autograd.grad(preds, inputs, grad_outputs=ones, create_graph=True)[0]
    delta = grad[:, 0]
    # Gamma: derivative of delta with respect to S
    gamma = torch.autograd.grad(delta, inputs, grad_outputs=torch.ones_like(delta), create_graph=True)[0][:, 0]

    preds_np = preds.detach().cpu().numpy()
    delta_np = delta.detach().cpu().numpy()
    gamma_np = gamma.detach().cpu().numpy()

    analytic = bs_greeks(S_np, K, T, t_np, sigma_np, r)
    analytic_delta = analytic['delta']
    analytic_gamma = analytic['gamma']

    price_mae = np.mean(np.abs(preds_np - target_price))
    price_rmse = math.sqrt(np.mean((preds_np - target_price)**2))
    delta_mae = np.mean(np.abs(delta_np - analytic_delta))
    gamma_mae = np.mean(np.abs(gamma_np - analytic_gamma))

    evaluation_summary = {
        'N_eval': int(N_eval),
        'price_mae': float(price_mae),
        'price_rmse': float(price_rmse),
        'delta_mae': float(delta_mae),
        'gamma_mae': float(gamma_mae),
    }

    print(json.dumps(evaluation_summary, indent=2))


{
  "N_eval": 5000,
  "price_mae": 31.619967691443946,
  "price_rmse": 36.85272648283381,
  "delta_mae": 0.6316764053012146,
  "gamma_mae": 0.005521820800736746
}


In [9]:

# 7) PDE residual + surface diagnostics
if not TORCH_OK or model is None:
    residual_stats = None
    print('Skipping PDE residual diagnostics (model unavailable).')
else:
    from src.losses import compute_pde_residual

    device = next(model.parameters()).device
    G = CONFIG['residual_grid']
    S_grid = torch.linspace(20, 200, G, device=device)
    sigma_grid = torch.linspace(0.05, 0.6, G, device=device)
    S_mesh, sigma_mesh = torch.meshgrid(S_grid, sigma_grid, indexing='ij')
    t_mesh = torch.full_like(S_mesh, snapshot_t)

    resid = compute_pde_residual(
        model,
        S_mesh.flatten(),
        t_mesh.flatten(),
        sigma_mesh.flatten(),
        r=r,
    )
    resid_np = resid.detach().cpu().numpy().reshape(G, G)

    residual_stats = {
        'mean': float(resid_np.mean()),
        'std': float(resid_np.std()),
        'max_abs': float(np.max(np.abs(resid_np))),
    }
    print(json.dumps(residual_stats, indent=2))

    plt.figure(figsize=(6,4))
    plt.contourf(S_grid.cpu().numpy(), sigma_grid.cpu().numpy(), resid_np.T, levels=40, cmap='coolwarm')
    plt.colorbar(label='PDE residual')
    plt.xlabel('S')
    plt.ylabel('sigma')
    plt.title('PDE residual heatmap (stress grid)')
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'stress_test' / 'pde_residual_heatmap.png', dpi=200)

    with torch.no_grad():
        coords = torch.stack([
            S_mesh.flatten(),
            t_mesh.flatten(),
            sigma_mesh.flatten(),
        ], dim=1)
        surface = model(coords).detach().cpu().numpy().reshape(G, G)
    plt.figure(figsize=(6,4))
    plt.contourf(S_grid.cpu().numpy(), sigma_grid.cpu().numpy(), surface.T, levels=40, cmap='viridis')
    plt.colorbar(label='Price')
    plt.xlabel('S')
    plt.ylabel('sigma')
    plt.title('PINN price surface (stress grid)')
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / 'stress_test' / 'surface_grid.png', dpi=200)


ValueError: cannot reshape array of size 2560000 into shape (40,40)

In [ ]:

# 8) Consolidated summary + optional JSON export
summary_payload = {
    'timestamp': datetime.utcnow().isoformat() + 'Z',
    'config': CONFIG,
    'evaluation': evaluation_summary,
    'residual': residual_stats,
    'fd_delta_mae': float(fd_delta_mae),
    'fd_gamma_mae': float(fd_gamma_mae),
    'mc_delta_mean_abs_error': float(mc_df['abs_err'].mean()) if 'mc_df' in globals() else None,
}

print(json.dumps(summary_payload, indent=2))

summary_path = RESULTS_DIR / 'stress_test_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary_payload, f, indent=2)
print(f"Wrote summary to {summary_path}")
